In [5]:
!mkdir -p ~/.ssh
!cp /content/drive/My\ Drive/ssh_key ~/.ssh/id_rsa
!chmod 600 ~/.ssh/id_rsa


In [6]:
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts


# github.com:22 SSH-2.0-babeld-4909cb0f2


In [7]:
!ssh -T git@github.com


Hi AmrMSharafeldin! You've successfully authenticated, but GitHub does not provide shell access.


In [8]:
!git clone git@github.com:arvinbm/SUPPORT2_DATA_MINING.git


Cloning into 'SUPPORT2_DATA_MINING'...
remote: Enumerating objects: 12732, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 12732 (delta 130), reused 226 (delta 122), pack-reused 12493 (from 1)
Receiving objects: 100% (12732/12732), 96.05 MiB | 9.03 MiB/s, done.
Resolving deltas: 100% (1393/1393), done.


In [1]:
%cd SUPPORT2_DATA_MINING

/content/SUPPORT2_DATA_MINING


In [3]:
!pip install -r requirements.txt
!pip install pytorch-tabular

In [4]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.models import NodeConfig
from pytorch_tabular import TabularModel
from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig
from sklearn.model_selection import train_test_split
import pandas as pd
from data_preprocessing import get_processed_data
import yaml
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
processed_data, labels = get_processed_data(None)
y = labels['class_encoded']
train_data, test_data = train_test_split(processed_data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Combine train_data, val_data, and test_data with their respective targets
train = train_data.copy()
train["class"] = y.loc[train_data.index]

val = val_data.copy()
val["class"] = y.loc[val_data.index]

test = test_data.copy()
test["class"] = y.loc[test_data.index]

# Define column names for numerical and categorical features
num_col_names = [col for col in train.columns if train[col].dtype in ["float64", "int64"] and col != "class"]
cat_col_names = [col for col in train.columns if train[col].dtype == "object"]

### For a basic FeedForwad Category Embedding Model.

In [6]:
# Define DataConfig
data_config = DataConfig(
    target=["class"],  # Target should always be a list
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)

# Define TrainerConfig
trainer_config = TrainerConfig(
    auto_lr_find=False,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=100,
    #early_stopping=None,  # Disable early stopping
)

# Define OptimizerConfig
optimizer_config = OptimizerConfig()

# Define ModelConfig
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="1024-512-512",  # Number of nodes in each layer
    activation="LeakyReLU",  # Activation function between layers
    learning_rate=1e-3,
)

# Initialize the TabularModel
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

# Train the model
tabular_model.fit(train=train, validation=val)
result = tabular_model.evaluate(test)
pred_df = tabular_model.predict(test)
tabular_model.save_model("examples/basic")
loaded_model = TabularModel.load_model("examples/basic")


INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/tabular_datamodule.py:368: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.86715299 -0.86715299  1.15319905 ...  1.15319905  1.15319905
  1.15319905]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, self.config.continuous_cols] = self.scaler.fit_transform(
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/tabular_datamodule.py:368: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.86715299  0.86715299 -1.15319905 ... -1.15319905 -1.15319905
 -1.15319905]' has dty

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  842 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │    106 │
│ 2 │ head             │ LinearHead                │  1.5 K │
│ 3 │ loss             │ CrossEntropyLoss          │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 844 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 844 K                                                                                                
Total estimated model params size (MB): 3

Output()

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (6)
is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you 
want to see logs for the training epoch.

INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/utils/python_utils.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please op

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6979681253433228     │
│         test_loss         │    0.6254205703735352     │
└───────────────────────────┴───────────────────────────┘

/usr/local/lib/python3.10/dist-packages/pytorch_tabular/tabular_datamodule.py:372: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.86715299 -0.86715299 -0.86715299 ...  1.15319905  1.15319905
 -0.86715299]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, self.config.continuous_cols] = self.scaler.transform(data.loc[:, self.config.continuous_cols])
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/tabular_datamodule.py:372: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.86715299  0.86715299  0.86715299 ... -1.15319905 -1.15319905
  0.86715299]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, self.config.continuous_cols] = self.scaler.transform(data.loc[:, self.config.continuous_cols])
/usr/local/lib/python3.10/dist-packages/pytorc

In [8]:
from sklearn.metrics import classification_report, accuracy_score

true_labels = test["class"].values
predicted_labels = pred_df["prediction"].values

report = classification_report(true_labels, predicted_labels)

accuracy = accuracy_score(true_labels, predicted_labels)

print("Classification Report:\n", report)
print(f"Accuracy: {accuracy:.2f}")


Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.60      0.63       594
           1       0.82      0.81      0.81       474
           2       0.65      0.71      0.68       753

    accuracy                           0.70      1821
   macro avg       0.71      0.70      0.71      1821
weighted avg       0.70      0.70      0.70      1821

Accuracy: 0.70


##Neural Oblivious Decision Ensembles (NODE)

To avoid session out of RAM memroy crushed. Please run this on T4 GPU

In [11]:
# Define DataConfig
data_config = DataConfig(
    target=["class"],  # Target should always be a list
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)

# Define TrainerConfig
trainer_config = TrainerConfig(
    auto_lr_find=False,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=128,
    max_epochs=100,
    #early_stopping=None,  # Disable early stopping
)

# Define OptimizerConfig
optimizer_config = OptimizerConfig()

# Define ModelConfig
node_config = NodeConfig(
    num_layers=1,  # Number of Oblivious Decision Tree Layers in the Dense Architecture
    num_trees=2048,  # Number of Oblivious Decision Trees in each layer
    depth=6,  # Depth of the individual Oblivious Decision Trees
    choice_function="sparsemax",  # Sparse probability distribution for feature selection
    additional_tree_output_dim=3,  # Additional output dimensions for intermediate layers
    task="classification",
)

# Initialize the TabularModel
tabular_model = TabularModel(
    data_config=data_config,
    model_config=node_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

# Train the model
tabular_model.fit(train=train, validation=val)
result = tabular_model.evaluate(test)
pred_df = tabular_model.predict(test)
tabular_model.save_model("examples/basic")
loaded_model = TabularModel.load_model("examples/basic")

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/tabular_datamodule.py:368: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.86715299 -0.86715299  1.15319905 ...  1.15319905  1.15319905
  1.15319905]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, self.config.continuous_cols] = self.scaler.fit_transform(
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/tabular_datamodule.py:368: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.86715299  0.86715299 -1.15319905 ... -1.15319905 -1.15319905
 -1.15319905]' has dty

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ NODEBackbone     │  1.5 M │
│ 1 │ _embedding_layer │ Embedding1dLayer │    106 │
│ 2 │ _head            │ Lambda           │      0 │
│ 3 │ loss             │ CrossEntropyLoss │      0 │
└───┴──────────────────┴──────────────────┴────────┘

Trainable params: 1.5 M                                                                                            
Non-trainable params: 769                                                                                          
Total params: 1.5 M                                                                                                
Total estimated model params size (MB): 5

Output()

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches 
(46) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if
you want to see logs for the training epoch.

INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/utils/python_utils.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please op

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7517847418785095     │
│         test_loss         │    0.5773446559906006     │
└───────────────────────────┴───────────────────────────┘

/usr/local/lib/python3.10/dist-packages/pytorch_tabular/tabular_datamodule.py:372: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.86715299 -0.86715299 -0.86715299 ...  1.15319905  1.15319905
 -0.86715299]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, self.config.continuous_cols] = self.scaler.transform(data.loc[:, self.config.continuous_cols])
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/tabular_datamodule.py:372: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.86715299  0.86715299  0.86715299 ... -1.15319905 -1.15319905
  0.86715299]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, self.config.continuous_cols] = self.scaler.transform(data.loc[:, self.config.continuous_cols])
/usr/local/lib/python3.10/dist-packages/pytorc

In [12]:
from sklearn.metrics import classification_report, accuracy_score

true_labels = test["class"].values
predicted_labels = pred_df["prediction"].values

report = classification_report(true_labels, predicted_labels)

accuracy = accuracy_score(true_labels, predicted_labels)

print("Classification Report:\n", report)
print(f"Accuracy: {accuracy:.2f}")


Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.70      0.70       594
           1       0.86      0.83      0.85       474
           2       0.72      0.74      0.73       753

    accuracy                           0.75      1821
   macro avg       0.76      0.76      0.76      1821
weighted avg       0.75      0.75      0.75      1821

Accuracy: 0.75
